In [1]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("train(house).csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1168, 79), (292, 79), (1168, 2), (292, 2))

In [35]:
# EDA

X_train.isnull().sum()
X_train.head()
# print(X_train.info())

# 결측값 채우기
column = X_train.columns
for c in column:
    if X_train[c].dtype == 'O':
        X_train[c] = X_train[c].fillna("-")
        X_test[c] = X_test[c].fillna("-")
    else:
        X_train[c] = X_train[c].fillna(0)
        X_test[c] = X_test[c].fillna(0)

X_test

# drop도 좋아보인다.

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1380,30,RL,45.0,8212,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,-,-,-,0,6,2010,WD,Normal
520,190,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,-,-,-,0,8,2008,WD,Normal
1175,50,RL,85.0,10678,Pave,-,Reg,Lvl,AllPub,Inside,...,0,0,-,-,-,0,4,2007,WD,Normal
351,120,RL,0.0,5271,Pave,-,IR1,Low,AllPub,Inside,...,184,0,-,-,-,0,12,2006,WD,Abnorml
1335,20,RL,80.0,9650,Pave,-,Reg,Lvl,AllPub,Inside,...,0,0,-,-,-,0,4,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,90,RL,94.0,9400,Pave,-,Reg,Low,AllPub,Corner,...,0,0,-,-,-,0,4,2010,WD,Normal
1039,180,RM,21.0,1477,Pave,-,Reg,Lvl,AllPub,Inside,...,0,0,-,-,-,0,4,2009,WD,Normal
61,75,RM,60.0,7200,Pave,-,Reg,Lvl,AllPub,Inside,...,0,0,-,-,-,0,3,2007,WD,Normal
1395,60,RL,88.0,12665,Pave,-,IR1,Lvl,AllPub,Inside,...,0,0,-,-,-,0,2,2007,WD,Normal


In [40]:
drop_y_train = y_train.drop('Id', axis = 1)
drop_y_test = y_test.drop('Id', axis = 1)

In [41]:
# 범주형 변수 변경
from sklearn.preprocessing import LabelEncoder # 트리 모델인 경우

pre = LabelEncoder()

for c in column:
    if X_train[c].dtype == 'O':
        X_train[c] = pre.fit_transform(X_train[c])
        X_test[c] = pre.fit_transform(X_test[c])

X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
81,120,4,32.0,4500,1,0,3,3,0,2,...,0,0,0,0,0,0,3,2006,8,4
1418,20,3,71.0,9204,1,0,3,3,0,4,...,0,0,0,0,0,0,8,2008,0,4
1212,30,3,50.0,9340,1,0,3,3,0,4,...,0,0,0,0,0,0,8,2009,8,4
588,20,3,65.0,25095,1,0,0,2,0,4,...,60,0,0,0,0,0,6,2009,8,5
251,120,4,44.0,4750,1,0,0,1,0,4,...,153,0,0,0,0,0,12,2007,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,60,3,90.0,10800,1,0,3,3,0,4,...,0,0,0,0,0,0,6,2008,8,4
1152,20,3,90.0,14115,1,0,0,3,0,4,...,0,0,0,0,0,0,7,2006,8,0
57,60,3,89.0,11645,1,0,0,3,0,0,...,0,0,0,0,0,0,8,2006,8,4
1365,60,1,0.0,7500,1,0,3,3,0,4,...,0,0,0,0,0,0,1,2010,8,4


In [46]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, drop_y_train)

C:\Users\rapa\anaconda3\envs\data_analysis\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [51]:
# 평가 수행하기 mean_squared_error
from sklearn.metrics import root_mean_squared_error, r2_score

train_pred = model.predict(X_train)
train_pred

train_rmse = root_mean_squared_error(train_pred, drop_y_train)
train_r2 = r2_score(train_pred, drop_y_train)

print(train_rmse, train_r2)

11998.292946522308 0.9747106198907436


In [58]:
y_test

,Id,SalePrice
1380,1381,58500
520,521,106250
1175,1176,285000
351,352,190000
1335,1336,167900
...,...,...
1283,1284,139000
1039,1040,80000
61,62,101000
1395,1396,281213


In [59]:
# 테스트 결과 저장하기

test_pred = model.predict(X_test)

test_rmse = root_mean_squared_error(test_pred, drop_y_test)
test_r2 = r2_score(test_pred, drop_y_test)

print(test_rmse, test_r2)

data = pd.DataFrame({"Id": y_test['Id'], "SalePrice":test_pred})

# data.to_csv("result.csv", index = False)

26109.203893424667 0.8404573436588051


,Id,SalePrice
1380,1381,90712.02
520,521,104501.16
1175,1176,337250.90
351,352,204414.08
1335,1336,169910.90
...,...,...
1283,1284,149170.17
1039,1040,85924.94
61,62,94202.46
1395,1396,321122.52
